## **SoccerTwos**

##**Step 0: Install MLAgents and download the correct executable**


To be able to train our agents correctly and push to the Hub, we need to install ML-Agents


In [20]:
%%capture
# Clone the repository
!git clone --depth 1 https://github.com/Unity-Technologies/ml-agents

When the cloning is done (it takes 2.63 GB), we go inside the repository and install the package

In [21]:
%%capture
# Go inside the repository and install the package
%cd ml-agents
!pip3 install -e ./ml-agents-envs
!pip3 install -e ./ml-agents

### Download and move the environment zip file in `./training-envs-executables/linux/`
- Our environment executable is in a zip file.
- We need to download it and place it to `./training-envs-executables/linux/`
- We use a linux executable because we use colab, and colab machines OS is Ubuntu (linux)

In [ ]:
# Here, we create training-envs-executables and linux
!mkdir ./training-envs-executables
!mkdir ./training-envs-executables/linux

Download the file SoccerTwos.zip from https://drive.google.com/file/d/1KuqBKYiXiIcU4kNMqEzhgypuFP5_45CL using `wget`. Check out the full solution to download large files from GDrive [here](https://bcrf.biochem.wisc.edu/2021/02/05/download-google-drive-files-using-wget/)


In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://drive.google.com/file/d/1KuqBKYiXiIcU4kNMqEzhgypuFP5_45CL' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1KuqBKYiXiIcU4kNMqEzhgypuFP5_45CL" -O ./training-envs-executables/linux/SoccerTwos.zip && rm -rf /tmp/cookies.txt

We unzip the executable.zip file

In [24]:
%%capture
!unzip -d ./training-envs-executables/linux/ ./training-envs-executables/linux/SoccerTwos.zip

Make sure your file is accessible

In [25]:
!chmod -R 755 ./training-envs-executables/linux/SoccerTwos.x86_64

###**Step 1 : Define the SoccerTwos config file**
- In ML-Agents, you define the **training hyperparameters into config.yaml files.**

There are multiple hyperparameters. To know them better, you should check for each explanation with [the documentation](https://github.com/Unity-Technologies/ml-agents/blob/release_20_docs/docs/Training-Configuration-File.md)


So you need to create a `SoccerTwos.yaml` config file in ./config/poca/SoccerTwos.yaml

We'll give you here a first version of this config (to copy and paste into your `SoccerTwos.yaml file`), **but you should modify it**.



In [ ]:
behaviors:
  SoccerTwos:
    trainer_type: poca
    hyperparameters:
      batch_size: 2048
      buffer_size: 20480
      learning_rate: 0.0003
      beta: 0.005
      epsilon: 0.2
      lambd: 0.95
      num_epoch: 3
      learning_rate_schedule: constant
    network_settings:
      normalize: false
      hidden_units: 512
      num_layers: 2
      vis_encode_type: simple
    reward_signals:
      extrinsic:
        gamma: 0.99
        strength: 1.0
    keep_checkpoints: 5
    max_steps: 5000000
    time_horizon: 1000
    summary_freq: 10000
    self_play:
      save_steps: 50000
      team_change: 200000
      swap_steps: 2000
      window: 10
      play_against_latest_model_ratio: 0.5
      initial_elo: 1200.0

### **Step 2: Train the agent**

To train our agent, we just need to **launch mlagents-learn and select the executable containing the environment.**

We define four parameters:

1. `mlagents-learn <config>`: the path where the hyperparameter config file is.
2. `--env`: where the environment executable is.
3. `--run_id`: the name you want to give to your training run id.
4. `--no-graphics`: to not launch the visualization during the training.


Train the model and use the `--resume` flag to continue training in case of interruption.

> It will fail first time if and when you use `--resume`, try running the block again to bypass the error.



In [ ]:
!mlagents-learn ./config/poca/SoccerTwos.yaml --env=./training-envs-executables/linux/SoccerTwos.x86_64 --run-id="SoccerTwos5" --no-graphics

##Step 3:  Push the agent to the 🤗 Hub

In [ ]:
!git config --global credential.helper store
!huggingface-cli login

And we define 4 parameters:

1. `--run-id`: the name of the training run id.
2. `--local-dir`: where the agent was saved, it’s results/<run_id name>, so in my case results/First Training.
3. `--repo-id`: the name of the Hugging Face repo you want to create or update. It’s always <your huggingface username>/<the repo name>
If the repo does not exist **it will be created automatically**
4. `--commit-message`: since HF repos are git repository you need to define a commit message.

In [ ]:
# !mlagents-push-to-hf --run-id="Soccer2Training" --local-dir="./results/SoccerTwos" --repo-id="JanainaNeres/SoccerTwos" --commit-message="SoccerTwos"